---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [7]:
def answer_one():
        
    # Your Code Here
    G = nx.read_edgelist("Employee_Movie_Choices.txt",delimiter = "	", create_using=nx.Graph())
    
    return G # Your Answer Here
G = answer_one()
G.number_of_nodes()
G.number_of_edges()

24

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [23]:
def answer_two():
    
    # Your Code Here
    df = pd.read_csv("Employee_Movie_Choices.txt",delimiter = "	", header = 0, names = ['employee', 'movie'])
    employee_names = df['employee'].unique()
    movie_names = df['movie'].unique()
    node_dict = {}
    for employee in employee_names:
        node_dict[employee] = 'employee'
    for movie in movie_names:
        node_dict[movie] = 'movie'
    
    G = nx.read_edgelist("Employee_Movie_Choices.txt",delimiter = "	", create_using=nx.Graph())
    nx.set_node_attributes(G, values = node_dict, name = 'type')

    
    return G# Your Answer Here

G = answer_two()
G.nodes(data = True)

[('Andy', {'type': 'employee'}),
 ('Anaconda', {'type': 'movie'}),
 ('Mean Girls', {'type': 'movie'}),
 ('The Matrix', {'type': 'movie'}),
 ('Claude', {'type': 'employee'}),
 ('Monty Python and the Holy Grail', {'type': 'movie'}),
 ('Snakes on a Plane', {'type': 'movie'}),
 ('Frida', {'type': 'employee'}),
 ('The Shawshank Redemption', {'type': 'movie'}),
 ('The Social Network', {'type': 'movie'}),
 ('Georgia', {'type': 'employee'}),
 ('Joan', {'type': 'employee'}),
 ('Forrest Gump', {'type': 'movie'}),
 ('Kung Fu Panda', {'type': 'movie'}),
 ('Lee', {'type': 'employee'}),
 ('Pablo', {'type': 'employee'}),
 ('The Dark Knight', {'type': 'movie'}),
 ('Vincent', {'type': 'employee'}),
 ('The Godfather', {'type': 'movie'})]

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [27]:
from networkx.algorithms import bipartite
def answer_three():
        
    # Your Code Here
    df = pd.read_csv("Employee_Movie_Choices.txt",delimiter = "	", header = 0, names = ['employee', 'movie'])
    employee_names = df['employee'].unique()
    movie_names = df['movie'].unique()
    node_dict = {}
    for employee in employee_names:
        node_dict[employee] = 'employee'
    for movie in movie_names:
        node_dict[movie] = 'movie'
    
    G = nx.read_edgelist("Employee_Movie_Choices.txt",delimiter = "	", create_using=nx.Graph())
    nx.set_node_attributes(G, values = node_dict, name = 'type')
    
    B = bipartite.weighted_projected_graph(G, employee_names, ratio=False)
    return B# Your Answer Here
B = answer_three()
B.edges(data=True)

[('Andy', 'Joan', {'weight': 1}),
 ('Andy', 'Georgia', {'weight': 1}),
 ('Andy', 'Lee', {'weight': 1}),
 ('Andy', 'Claude', {'weight': 1}),
 ('Andy', 'Pablo', {'weight': 1}),
 ('Andy', 'Frida', {'weight': 1}),
 ('Claude', 'Georgia', {'weight': 3}),
 ('Frida', 'Vincent', {'weight': 2}),
 ('Frida', 'Pablo', {'weight': 2}),
 ('Joan', 'Lee', {'weight': 3}),
 ('Pablo', 'Vincent', {'weight': 1})]

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [34]:
from networkx.algorithms import bipartite
def answer_four():
        
    # Your Code Here
    # 1. Read data files
    relationship = pd.read_csv("Employee_Relationships.txt",delimiter = "	", names = ['employee1', 'employee2', 'score'])
    movies = pd.read_csv("Employee_Movie_Choices.txt",delimiter = "	", header = 0, names = ['employee', 'movie'])
    
    # 2. Get common movies
    G = nx.read_edgelist("Employee_Movie_Choices.txt",delimiter = "	", create_using=nx.Graph())
    print(bipartite.is_bipartite(G))
    B = bipartite.weighted_projected_graph(G, movies['employee'].unique(), ratio=False)
    movie_commons = B.edges(data = True)
    
    # 3. Insert common movie data into relationship df (if no movie, movie score = 0)
    movie_commons_col = []
    for r in range(relationship.shape[0]):
        e1, e2 = relationship.loc[r, 'employee1'], relationship.loc[r, 'employee2']
        if B.has_edge(e1, e2):
            movie_commons_col.append(B.get_edge_data(e1, e2)['weight'])
        else:
            movie_commons_col.append(0)
    relationship['movies'] = movie_commons_col
    
    return relationship['score'].corr(relationship['movies'])# Your Answer Here
answer_four()

True


0.78839622217334748